# Assignment - 8 Description

In PyQuil, implement the Deutsch-Jozsa algorithm and the Bernstein-Vazirani algorithm.  Write detailed comments in the code about why it works.  Run the programs on the simulator.  Write a report that covers the following three points.

1. Design

    * Present the design of how you implemented the black-box function U_f.  Assess how visually neat and easy to read it is.
    * Present the design for how you prevent the user of U_f from accessing the implementation of U_f.  Assess how well you succeeded.
    * Present the design of how you parameterized the solution in n.
    * Discuss the number of lines and percentage of code that your two programs share.  Assess how well you succeeded in reusing code from one program to the next.


2. Evaluation

    * Discuss your effort to test the two programs and present results from the testing.  Discuss whether different cases of U_f lead to different execution times.
    * What is your experience with scalability as n grows?  Present a diagram that maps n to execution time.

3. Instructions

    * Present a README file that describes how to input the function f, how to run the program, and how to understand the output.
    * Submit three files, one for each program and one with the report.

# Bernstein- Vazirani: The Problem

1. Input: a function f: {0,1}^n → {0,1}.
2. Assumption: f(x) = a * x + b.
3. Output: a, b.

Notation: {0,1}^n is the set of bit strings of length n, a is an unknown bit string of length n, * is inner product mod 2, + is addition mod 2, and b is an unknown single bit.

# Headers Required

In [1]:
import numpy as np
import time
import itertools as itert
from pyquil import Program
from pyquil.quil import DefGate
from pyquil.gates import *
from pyquil import get_qc
from pyquil.quilatom import unpack_qubit

# Defining our current function


#### This is a random function generator which gives a, b for current running instance.

#### This helps ensure that our test cases are not biased

In [2]:
def function_generator(n):
    # a- binary string of length n
    a = np.random.randint(low=0, high=2, size=n)
    print ("This is the (randomly chosen) value of a: ", a)
    # b- single bit binary digit
    b = np.random.randint(low=0, high=2, size=1)
    print ("This is the (randomly chosen) value of b: ", b)
    return a,b

# Solving for 'b' classically 

#### As per class notes, Week 3 -> Palsberg; Algorithms: Bernstein-Vazirani File, we solve for 'b' classically and 'a' using quantum computing

In [3]:

def solveB (n,a,b):
    zero_vec=[0]*n
    curr_dot_product=0
    for i in range(0,n):
        curr_dot_product += a[i] * zero_vec[i]
    curr_dot_product %= 2
    curr_dot_product+=b
    curr_dot_product %= 2
    print("On solving b classically we get, b = ", curr_dot_product)

# Creating the Uf Matrix

#### The code differs from the Deutsch Jozsa algorithm only in terms of parameters and function application.

In [4]:
#this function creates the blackbox oracle Uf matrix for a given function f that is parameterised for input size n

def createUf(n,a,b):
    
    #this dictionary represents the correspondence between bit combinations and Uf indices
    indices_dict = {}
    counter = 0
    Uf = np.zeros((2**(n+1),2**(n+1)))

    bfx = 0

    # the for loops given below form the different pattern combinations that we need. The mapping between these combination and Uf matrix indices is stored in indices_dict
    for i in range(2**(n+1)):
        binary_form = ('{0:0'+str(n+1)+'b}').format(i) 
        indices_dict[binary_form] = counter
        counter += 1

    #print(indices_dict)
    #we then iterate through this pattern dictionary
    for key,val in indices_dict.items():
        
        #input to the function is the first n bits of the elements (bit patterns) from the dictionary
        x = key[0:n]
        #print("X= ", x)
        curr_dot_product = 0
        #for k, expand_q in enumerate(x):
            #print(x)
            #print(k)
            #curr_dot_product += a[k] * int(expand_q)
        for i in range(0,n):
            curr_dot_product += a[i] * int(x[i])
        curr_dot_product %= 2
        curr_dot_product= int(curr_dot_product) + int(b)
        curr_dot_product %= 2
        
        # f from current string
        
        fx= str(curr_dot_product) 
        
        #fx represents the output of function f given the input x
        #fx = str(f(x))
        
        #b is the last bit of the bit pattern in the dictionary item
        b1 = key[n]
        
        #below we have the (f(x) + b) mod 2
        if(b1==fx):
            bfx = '0'
        else:
            bfx = '1'
            # print(bfx)
        
        #the final bit string is the concatenation of the input x and bfx
        target = x + bfx
        # print('t',target)
        
        #using indices_dict we can now find the index that corresponds to this output
        target_index = indices_dict[target]
        # print(val,target_index)
        
        #now using the target indiex we can create a bit pattern with all 0s and 1 at the target index position
        Uf[val][target_index] = 1
        
    return Uf

# Creating our main circuit

* This circuit will be used to find 'a'. [Displayed in Results]

* Implementation similar to Deutsch Jozsa except in terms of params and some function calls.

* After measurement, the last bit can be discarded as its the result from the helper bit. All other bits denote 'a'

* for n=1, circuit should look like

###### |0> - H - | Uf |- H - M -
###### |1> - H - |___|- x




In [5]:
#def runMainCircuit(function,n,nTrials):
def runMainCircuit():
    
    
    # Taking the input n from the user
    n= int(input("Enter length of function input [Don't include helper bit in n and ONLY Integer Values Allowed]: "))
    
    # one extra bit as our helper bit 
    n = n+1
    
    # defining start of PyQuil program
    p = Program()
    
    qc_name = "{}q-qvm".format(n)
    
    # Get our QuantumComputer instance, with a Quantum Virutal Machine (QVM) backend
    
    qc = get_qc(qc_name)
    
    # Time taken by program needs to be checked.
    start = time. time()
    
    # generating my function
    print ("\n\n-----------This function is not accessible to user-----------")
    print ("Function generator output shown for verification/ proof of correctness ")
    
    curr_a, curr_b = function_generator(n)
    print ("-----------Restricted section over-----------\n\n")
    #classically solving b
    
    solveB(n-1,curr_a,curr_b)
    
    # setting last qubit to 1
    p += X(n-1)
    
    # adding Hadamard gates to all qubits
    for i in range(0,n):
        p += H(i)

    # creating our Uf matrix 
    UfMatrix = createUf(n-1,curr_a,curr_b) 
    GateName = "UF_GATE_BV"
    
    print("Matrix obtained: \n", UfMatrix)
    
    #defining a gate using its name and matrix
    
    uf_gate_definition = DefGate(GateName, UfMatrix)
    qubits = [unpack_qubit(i) for i in range(0,n)]
    
    # adding Uf gate
    
    p+=Program(uf_gate_definition,Gate(name=GateName, params=[],qubits=qubits))
    
    # helper bit does not require H gate. Result is treated as trash/ garbage
    
    for i in range(0,n-1):
        p += H(i)
        
    # transformed matrix
    
    #print("p at this point:")
    
    #print(p)
    
    # measurement result    
    results = qc.run_and_measure(p, trials=1)
    
    print("Results: ")
    print(results)
    end = time. time()
    print("Time taken by program: ", end-start)

# Bernstein-Vazirani - Test Case Examples

## Trial 1: n =1

In [6]:
runMainCircuit()

Enter length of function input [Don't include helper bit in n and ONLY Integer Values Allowed]: 1


-----------This function is not accessible to user-----------
Function generator output shown for verification/ proof of correctness 
This is the (randomly chosen) value of a:  [1 0]
This is the (randomly chosen) value of b:  [0]
-----------Restricted section over-----------


On solving b classically we get, b =  [0]
Matrix obtained: 
 [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]
Results: 
{0: array([1]), 1: array([0])}
Time taken by program:  0.3182108402252197


## Trial 2: n=2

In [7]:
runMainCircuit()

Enter length of function input [Don't include helper bit in n and ONLY Integer Values Allowed]: 2


-----------This function is not accessible to user-----------
Function generator output shown for verification/ proof of correctness 
This is the (randomly chosen) value of a:  [1 0 1]
This is the (randomly chosen) value of b:  [0]
-----------Restricted section over-----------


On solving b classically we get, b =  [0]
Matrix obtained: 
 [[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0.]]
Results: 
{0: array([1]), 1: array([0]), 2: array([0])}
Time taken by program:  0.5471782684326172


## Trial 3: n=3

In [12]:
runMainCircuit()

Enter length of function input [Don't include helper bit in n and ONLY Integer Values Allowed]: 3


-----------This function is not accessible to user-----------
Function generator output shown for verification/ proof of correctness 
This is the (randomly chosen) value of a:  [1 0 0 1]
This is the (randomly chosen) value of b:  [1]
-----------Restricted section over-----------


On solving b classically we get, b =  [1]
Matrix obtained: 
 [[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 

## Trial 4: n=4

In [10]:
runMainCircuit()

Enter length of function input [Don't include helper bit in n and ONLY Integer Values Allowed]: 4


-----------This function is not accessible to user-----------
Function generator output shown for verification/ proof of correctness 
This is the (randomly chosen) value of a:  [0 0 0 0 0]
This is the (randomly chosen) value of b:  [0]
-----------Restricted section over-----------


On solving b classically we get, b =  [0]
Matrix obtained: 
 [[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
Results: 
{0: array([0]), 1: array([0]), 2: array([0]), 3: array([0]), 4: array([1])}
Time taken by program:  2.09677791595459
